In [ ]:
from collections import defaultdict
import numpy as np

class HierarchicalHSCodeClassifier:
    def __init__(self, hs_graph, vector_db, llm):
        self.graph = hs_graph
        self.vector_db = vector_db
        self.llm = llm
        self.confidence_threshold = 0.65  # 조정 가능
        
    def classify(self, user_input, max_depth=5):
        """
        최상위부터 시작해서 트리를 내려가며 분류
        """
        path = []  # 선택 경로 기록
        current_level = "root"
        
        for depth in range(max_depth):
            # 현재 노드의 자식들 가져오기
            children = self.get_children(current_level)
            
            if not children:
                # 리프 노드 도달
                break
            
            # 다음 노드 선택
            selected, confidence, reason = self.select_next_node(
                user_input, 
                children, 
                current_level,
                path
            )
            
            path.append({
                "level": depth,
                "selected": selected,
                "confidence": confidence,
                "reason": reason
            })
            
            current_level = selected
            
            # 10자리 코드 도달
            if len(selected) >= 10:
                break
        
        return {
            "hs_code": current_level,
            "path": path,
            "final_confidence": path[-1]["confidence"] if path else 0
        }
    
    def select_next_node(self, user_input, children, parent_code, path_history):
        """
        자식 노드들 중에서 가장 적합한 것 선택
        핵심: "기타"는 마지막에 고려
        """
        
        # Step 1: 자식 노드를 "명시적" vs "기타"로 분리
        specific_children = [c for c in children if not self.graph[c].get("is_other")]
        other_children = [c for c in children if self.graph[c].get("is_other")]
        
        # Step 2: 명시적 노드들에 대해 점수 계산
        scores = {}
        evidence = {}
        
        for child_code in specific_children:
            score, ev = self.calculate_node_score(user_input, child_code)
            scores[child_code] = score
            evidence[child_code] = ev
        
        # Step 3: 최고 점수 확인
        if scores:
            best_code = max(scores, key=scores.get)
            best_score = scores[best_code]
            
            # Step 4: 신뢰도 충분하면 해당 노드 선택
            if best_score >= self.confidence_threshold:
                return best_code, best_score, evidence[best_code]
        
        # Step 5: 신뢰도 부족 → LLM에게 물어보기
        if specific_children:
            llm_decision = self.llm_decide_with_other(
                user_input,
                parent_code,
                specific_children,
                other_children,
                scores,
                evidence
            )
            
            return llm_decision["code"], llm_decision["confidence"], llm_decision["reason"]
        
        # Step 6: 명시적 노드가 없으면 "기타" 선택
        if other_children:
            return other_children[0], 0.5, "명시적 옵션 없음, 기타로 분류"
        
        raise ValueError("자식 노드가 없습니다")
    
    def calculate_node_score(self, user_input, node_code):
        """
        특정 노드에 대한 점수 계산
        품목분류사례 기반 유사도
        """
        
        # 이 노드 또는 하위 노드에 해당하는 분류사례들 검색
        relevant_cases = self.vector_db.search(
            query=user_input,
            filter={
                "hs_code": {
                    "$regex": f"^{node_code}"  # 해당 코드로 시작하는 모든 사례
                }
            },
            top_k=5
        )
        
        if not relevant_cases:
            # 해당 노드의 분류사례가 없으면 낮은 점수
            return 0.3, {"reason": "관련 분류사례 없음"}
        
        # 평균 유사도 점수
        avg_score = np.mean([case["score"] for case in relevant_cases])
        
        evidence = {
            "reason": f"{len(relevant_cases)}개 유사 사례 발견",
            "top_case": relevant_cases[0]["품목명"],
            "similarity": avg_score
        }
        
        return avg_score, evidence
    
    def llm_decide_with_other(self, user_input, parent_code, 
                               specific_children, other_children, 
                               scores, evidence):
        """
        LLM에게 명시적 옵션 vs "기타" 판단 요청
        """
        
        parent_name = self.graph[parent_code]["name"]
        
        # 프롬프트 구성
        options_text = []
        for child_code in specific_children:
            child_info = self.graph[child_code]
            score = scores.get(child_code, 0)
            ev = evidence.get(child_code, {})
            
            option = f"""
코드: {child_code}
명칭: {child_info['name']}
유사도 점수: {score:.2f}
근거: {ev.get('reason', '없음')}
"""
            if ev.get('top_case'):
                option += f"가장 유사한 사례: {ev['top_case']}\n"
            
            options_text.append(option)
        
        # "기타" 옵션 추가
        if other_children:
            for other_code in other_children:
                other_info = self.graph[other_code]
                options_text.append(f"""
코드: {other_code}
명칭: {other_info['name']} (기타 항목)
조건: 위의 명시적 항목들에 명확히 해당하지 않는 경우
""")
        
        prompt = f"""
품목 정보:
- 입력: {user_input}
- 현재 확정된 분류: {parent_name} ({parent_code})

다음 중 가장 적합한 세부 분류를 선택해주세요:

{chr(10).join(options_text)}

판단 기준:
1. 명시적 항목의 정의에 명확하고 확실하게 부합하는가?
2. 유사 분류사례가 충분히 뒷받침하는가?
3. 애매하거나 경계선상이면 "기타" 선택
4. 여러 명시적 항목에 걸치면 "기타" 선택
5. 명시적 항목의 정의를 벗어나면 "기타" 선택

반드시 다음 JSON 형식으로만 답변:
{{
    "selected_code": "선택한 코드",
    "confidence": 0.0-1.0 사이 숫자,
    "reason": "선택 이유 (한 문장)",
    "is_other": true 또는 false
}}
"""
        
        response = self.llm.generate(prompt)
        result = self.parse_llm_response(response)
        
        return {
            "code": result["selected_code"],
            "confidence": result["confidence"],
            "reason": result["reason"]
        }
    
    def get_children(self, node_code):
        """노드의 자식들 반환"""
        if node_code == "root":
            # 2자리 코드들 (01, 02, 03, ...)
            return [k for k in self.graph.keys() if len(k) == 2]
        
        node = self.graph.get(node_code)
        return node.get("children", []) if node else []
    
    def parse_llm_response(self, response):
        """LLM 응답 파싱"""
        import json
        try:
            return json.loads(response)
        except:
            # 파싱 실패 시 기본값
            return {
                "selected_code": "unknown",
                "confidence": 0.5,
                "reason": "파싱 실패",
                "is_other": False
            }

In [ ]:
# 함수 사용예시
# 초기화
classifier = HierarchicalHSCodeClassifier(
    hs_graph=hs_graph,
    vector_db=vector_db,
    llm=llm_client
)

# 분류 실행
result = classifier.classify("농장에서 기르는 노새")

print(result)
# {
#   "hs_code": "01019000",
#   "path": [
#       {
#           "level": 0,
#           "selected": "01",
#           "confidence": 0.95,
#           "reason": "15개 유사 사례 발견"
#       },
#       {
#           "level": 1, 
#           "selected": "0101",
#           "confidence": 0.88,
#           "reason": "8개 유사 사례 발견"
#       },
#       {
#           "level": 2,
#           "selected": "010190",  # ← 기타 선택됨
#           "confidence": 0.70,
#           "reason": "순종말(010121), 일반말(010129), 당나귀(010130)에 해당하지 않음. 노새는 기타 항목"
#       },
#       {
#           "level": 3,
#           "selected": "01019000",
#           "confidence": 0.70,
#           "reason": "리프 노드"
#       }
#   ],
#   "final_confidence": 0.70
# }